In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
coureurs = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Coureurs")
motor = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Motor")
chassis = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Chassis")
teams = pd.read_excel("acwe2023.xlsx", index_col="id", sheet_name="Teams")

races = ['Bahrein', 'Saudi-Arabië', 'Australië']

In [3]:
def calc_score_coureur(i, race):
    score = coureurs.loc[coureurs.index == i][race] * 0.5
    return score.values[0]

def calc_score_chassis(i, race):
    score = coureurs.loc[coureurs['chassis'] == i][race]
    return score.sum() * 0.2
    
def calc_score_motor(i, race):
    score = coureurs.loc[coureurs['motor'] == i][race]
    return score.sum() / (0.5*score.count()) * 0.3

In [4]:
df = pd.DataFrame(columns=['team', 'race', 'cat', 'score'])

for race in races:
    score_coureur1 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur1'] ])
    score_coureur2 = pd.Series([ calc_score_coureur(i, race) for i in teams['coureur2'] ])
    score_chassis  = pd.Series([ calc_score_chassis(i, race) for i in teams['chassis' ] ])
    score_motor    = pd.Series([ calc_score_motor  (i, race) for i in teams['motor'   ] ])
    
    df_coureur1 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur1', 'score' : score_coureur1})
    df_coureur2 = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'coureur2', 'score' : score_coureur2})
    df_chassis = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'chassis', 'score' : score_chassis})
    df_motor = pd.DataFrame({'team' : teams['naam'], 'race' : race, 'cat' : 'motor', 'score' : score_motor})
    
    df = pd.concat([df, df_coureur1, df_coureur2, df_chassis, df_motor])

In [13]:
fig = px.histogram(df.loc[df['race']=='Bahrein'], x="team", y="score", color="cat",
            title="Scores behaald in Bahrein",
            width=800, height=400,
            labels={ "score" : "Score", "cat" : "Categorie", "team" : "Team" },
            template="simple_white"
            )
fig.show()
fig.write_html("test_bar.html")

In [14]:
df_score_per_race = df.groupby(['race', 'team'])['score'].sum(numeric_only=True).unstack()#.reset_index()#.rename_axis(None, axis=1)
df_score_per_race.transpose()
df_score_per_race = df_score_per_race.reindex(races)

In [15]:
fig = px.line(df_score_per_race, template='simple_white')
fig.show()